# Logistic Regression

In this notebook, I will show a very simple classification using Logistic Regression. My goal here is to determine based on stats whether the player is defenseman of forward. As a NHL enthusiast I'm using NHL dataset from season 2016-2017. 

In [21]:
#Lets read in packages I'm going to use in this notebook. If you don't have them, install them using command for example:
# install.packages('Amelia') and remember to use quotes

library(readxl)
library(dplyr)
library(MLmetrics)
library(caTools)

Warning message:
"package 'MLmetrics' was built under R version 3.4.4"
Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall

Warning message:
"package 'caTools' was built under R version 3.4.4"

In [7]:
#Lets read in the data
NHL_16_17 <- read_excel("NHL_16_17.xls")
nhl <- NHL_16_17[2:890,]
colnames(nhl) <- nhl[1,]
nhl <- nhl[2:889,]
head(nhl)

Born,City,Pr/St,Cntry,Nat,Ht,Wt,DftYr,DftRd,Ovrl,...,1st,2nd,3rd,MGL,Injuries,CHIP,NMC,Status,Salary,Cap Hit
1988-04-30,Hamilton,ON,CAN,CAN,69,170,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,UFA,575000,575000
1987-02-25,Muskegon,MI,USA,USA,74,218,2005,2,42,...,0,0,1,18,"Lower body, Knee",932927,NTC,UFA,5500000,4250000
1993-09-23,Stockholm,NA,SWE,SWE,71,196,2012,2,37,...,0,0,1,NA,NA,NA,NA,RFA,842500,780833
1991-12-01,Johnston,RI,USA,USA,70,208,NA,NA,NA,...,0,0,1,15,"Lower body, Upper body",144970,NA,RFA,892500,792500
1992-04-30,Morristown,NJ,USA,USA,72,202,2010,5,140,...,0,1,0,NA,NA,NA,NA,UFA,625000,625000
1997-07-26,Rauma,NA,FIN,FIN,71,172,2015,2,35,...,5,1,4,NA,NA,NA,NA,RFA,925000,925000


Here you can see that we have many different columns such as Nationality, Height, Weight and Country. But in this case I'm narrowing down columns to five possible candidates: points, time on ice per game, position, penalty minutes and shifts:

In [8]:
nhl$Points <- as.numeric(nhl$PTS)
points <- nhl$Points
nhl$TimeOnIce <- nhl$`TOI/GP`
nhl$TimeOnIce <- as.numeric(unlist(nhl$TimeOnIce))
timeOnIce <- nhl$TimeOnIce
nhl$Position <- as.factor(nhl$Position)
position <- nhl$Position
penalty_mins <- as.numeric(nhl$PIM)
shifts <- as.numeric(nhl$Shifts)

Then I will combine these vector in order to create data matrix:

In [12]:
dataset <- cbind(points, timeOnIce, penalty_mins, shifts)
dataset <- as.data.frame(dataset)
dataset$position <- as.factor(position)
head(dataset)

points,timeOnIce,penalty_mins,shifts,position
0,8.56667,0,12,LW
21,16.65000,50,1397,LW/RW
2,12.33330,4,256,LW
5,10.23330,16,431,C
3,12.78330,2,117,LW
49,16.78330,26,1814,RW/LW


Allrighty, now that looks much better than before. No unnecessary columns considering this simple approach. Next I'm going to refactor the position column/feature so that outputs are simply forward and defenseman:

In [13]:
table(dataset$position)
dataset$position <- as.character(dataset$position)
dataset$position[grep("^D",dataset$position)] <- "Defenseman"
dataset$position[grep("\\C|\\LW|\\RW",dataset$position)] <- "Forward"

dataset$position <- as.factor(dataset$position)
dataset$position <- as.numeric(dataset$position)
dataset <- dataset %>%
  mutate(position = position - 1)
table(dataset$position) #0 = defenseman, 1 = forward


      C     C/D    C/LW  C/LW/C C/LW/RW    C/RW C/RW/LW       D    D/LW    D/RW 
    144       1      56       1       9      42       7     296       1       2 
     LW    LW/C LW/C/RW   LW/RW LW/RW/C      RW    RW/C RW/C/LW   RW/LW RW/LW/C 
     79      47      10      34       5      91      23       5      31       4 


  0   1 
299 589 

Above I created frequency tables for different positions in this dataset. Here we can see that many players are versatile so that they can play different positions. Total number of defensemen is 299 and total number of forwards is 589.

I intended to use missmap function that is part of the Amelia package in order to visualize if there are any missing values, but for some reason I can't get it to work in this notebook. But instead I will just use simple base-R command to find out if there are any missing values

In [18]:
sum(is.na(dataset))

[1] 0

## Next phase is to build our model

In order for you to recreate my results, I'm going to set seed so that your and mine outputs should match.

In [19]:
set.seed(123)

Next we are going to split data into training and test set. I will use 75% of the data as training set and 25% as the test set. We are going to create model using training set and after that we test our prediction with test set:

In [22]:
split = sample.split(dataset$position, SplitRatio = 0.75)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

Let's scale our numeric values so that different scales don't mess up our model:

In [23]:
training_set[-5] = scale(training_set[-5])
test_set[-5] = scale(test_set[-5])

Finally we get to create our Logistic Regression model. I'm going to use all our dataset columns (1 to 5) and I will create model using training_set. I will also print out summary of our model:

In [24]:
classifier = glm(formula = position ~ .,
                 family = binomial,
                 data = training_set)

summary(classifier)


Call:
glm(formula = position ~ ., family = binomial, data = training_set)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.9216  -0.1941   0.1663   0.4313   2.3847  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)    1.3063     0.1556   8.396   <2e-16 ***
points         3.7335     0.4218   8.852   <2e-16 ***
timeOnIce     -3.6117     0.3075 -11.745   <2e-16 ***
penalty_mins  -0.1545     0.1950  -0.792    0.428    
shifts        -0.5245     0.3361  -1.560    0.119    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 850.58  on 665  degrees of freedom
Residual deviance: 355.33  on 661  degrees of freedom
AIC: 365.33

Number of Fisher Scoring iterations: 6


Here we are able to see that not our features/columns were relevant to this model. Focus on the last column showing those p-values. Looks like that penalty_mins and shifts are useless features in order to classify player to forwards/defenseman. Threshold for statistical significance is 0.05 and those two last features are far from it. So I'm going to remove those two features in order to improve our model.

In [25]:
dataset <- subset(dataset, select = -c(penalty_mins, shifts))
head(dataset)

points,timeOnIce,position
0,8.56667,1
21,16.65000,1
2,12.33330,1
5,10.23330,1
3,12.78330,1
49,16.78330,1


Let's do again the train/test split and create the model like before, but this time with only three features. Take notice that when I'm scaling training and test sets, I'm using [,3] instead of [,5]:

In [27]:
split = sample.split(dataset$position, SplitRatio = 0.75)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

training_set[-3] = scale(training_set[-3])
test_set[-3] = scale(test_set[-3])

classifier = glm(formula = position ~ .,
                 family = binomial,
                 data = training_set)

summary(classifier)


Call:
glm(formula = position ~ ., family = binomial, data = training_set)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.1802  -0.1683   0.1510   0.3664   2.7471  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   1.3368     0.1612   8.294   <2e-16 ***
points        3.5314     0.3064  11.525   <2e-16 ***
timeOnIce    -4.0999     0.3255 -12.594   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 850.58  on 665  degrees of freedom
Residual deviance: 328.94  on 663  degrees of freedom
AIC: 334.94

Number of Fisher Scoring iterations: 6


Now we get to predict classes to player using our test set as new data (of course without position labels):

In [28]:
prob_pred = predict(classifier, type = 'response', newdata = test_set[-3])
y_pred = ifelse(prob_pred > 0.5, 1, 0)

In the previous cell, if the probability is more than 50% we assign that data point to forward and if it is less than 50% we assign it to defenseman. Next we create a Confusion Matrix to see quickly how our model works

In [29]:
ConfusionMatrix(y_pred, test_set[,3])

      y_pred
y_true   0   1
     0  61  14
     1   8 139

Here we can see that our model manages to assign defenseman as defenseman 61 times and it missclassifies it 14 times. Our model assigns forwards to right position 139 times and missclassifies player as forward 8 times. We can also create different metrics to see how well our model works:

In [30]:
Accuracy(y_pred, test_set[,3])
Recall(y_pred, test_set[,3])
F1_Score(y_pred, test_set[,3])

[1] 0.9009009

[1] 0.884058

[1] 0.8472222

In this case our model manages to predict right position to players about 9 times out of 10. Pretty good with such a simple model, right! There are also many ways to improve our model, but this is the short demonstration how Logistic Regression works with R.

# Thanks for reading and enjoy coding!